In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import json

 does the MPAA rating of a movie (G/PG/PG-13/R) affect how much revenue the movie generates?

In [80]:
data2000 = pd.read_csv('Data/final_tmdb_data_2000.csv.gz')
data2001 = pd.read_csv('Data/final_tmdb_data_2001.csv.gz')
data2000_copy = data2000.copy()
data2001_copy = data2001.copy()

In [81]:
df = pd.concat([data2000, data2001], axis=0)
df_copy = df.copy()
df = df.drop(labels=0, axis=0)
df.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
1,tt0113026,0.0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,NaN,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,62127.0,en,The Fantasticks,...,0.0,86.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Try to remember the first time magic happened,The Fantasticks,0.0,5.50,22.0,NaN
2,tt0113092,0.0,NaN,NaN,0.0,"[{'id': 878, 'name': 'Science Fiction'}]",NaN,110977.0,en,For the Cause,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The ultimate showdown on a forbidden planet.,For the Cause,0.0,5.10,8.0,NaN
3,tt0116391,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,442869.0,hi,Gang,...,0.0,152.0,"[{'english_name': 'Hindi', 'iso_639_1': 'hi', ...",Released,NaN,Gang,0.0,4.00,1.0,NaN
4,tt0118694,0.0,/n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg,NaN,150000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,843.0,cn,花樣年華,...,12854953.0,99.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,"Feel the heat, keep the feeling burning, let t...",In the Mood for Love,0.0,8.11,2084.0,PG
5,tt0118852,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,49511.0,en,Chinese Coffee,...,0.0,99.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,There's a fine line between friendship and bet...,Chinese Coffee,0.0,6.80,48.0,R


In [82]:
df['genres'] = df['genres'].astype(str)

In [83]:
def get_genre_name(x):
    x = x.replace("'",'"')
    x = json.loads(x)
    genres = []
    for genre in x:
        genres.append(genre['name'])
    return genres

In [84]:
df['genres_list'] = df['genres'].apply(get_genre_name)
df_explode = df.explode('genres_list')

In [85]:
unique_genres = df_explode['genres_list'].dropna().unique()

In [86]:
for genre in unique_genres:
    df[f"Genre_{genre}"] = df['genres'].str.contains(genre,regex =False)

In [87]:
df = df.drop(columns=['genres','genres_list'])

In [88]:
df['certification'] = df['certification'].str.strip()
df['certification'].value_counts(dropna=False)

NaN        1716
R           453
PG-13       182
NR           67
PG           63
G            24
NC-17         6
Unrated       1
-             1
Name: certification, dtype: int64

In [89]:
cert = {'UR':'NR',
             'Not Rated':'NR',
             'Unrated':'NR',
             '-':'NR',
             '10':np.nan,
             'ScreamFest Horror Film Festival':'NR'}
df['certification'] = df['certification'].replace(cert)
df['certification'].value_counts(dropna=False)

NaN      1716
R         453
PG-13     182
NR         69
PG         63
G          24
NC-17       6
Name: certification, dtype: int64

In [90]:
df.drop(df.loc[df['revenue']==0].index, inplace=True)

In [91]:
df['revenue'] = df['revenue'].astype(float)

In [92]:
cols = ['certification', 'revenue']

In [93]:
r = df.loc[df['certification'] == 'R', cols]
pg13 = df.loc[df['certification'] == 'PG-13', cols]
nr = df.loc[df['certification'] == 'NR', cols]
pg = df.loc[df['certification'] == 'PG', cols]
g = df.loc[df['certification'] == 'G', cols]

In [94]:
r_group = r['revenue']
pg13_group = pg13['revenue']
nr_group = nr['revenue']
pg_group = pg['revenue']
g_group = g['revenue']

In [104]:
r_outliers = np.abs(stats.zscore(r_group))>3
r_outliers.value_counts()

False    50
Name: revenue, dtype: int64

In [105]:
r_group = r_group[(np.abs(stats.zscore(r_group)) < 3)]

In [106]:
pg13_outliers = np.abs(stats.zscore(pg13_group))>3
pg13_outliers.value_counts()

False    30
True      1
Name: revenue, dtype: int64

In [107]:
pg13_group = pg13_group[(np.abs(stats.zscore(pg13_group)) < 3)]

In [108]:
nr_outliers = np.abs(stats.zscore(nr_group))>3

nr_outliers.value_counts()

False    2
Name: revenue, dtype: int64

In [109]:
pg_outliers = np.abs(stats.zscore(pg_group))>3

pg_outliers.value_counts()

False    8
Name: revenue, dtype: int64

In [110]:
g_outliers = np.abs(stats.zscore(g_group))>3

g_group_outliers.value_counts()

False    1
Name: revenue, dtype: int64

In [111]:
nr_group.value_counts().sum()

2

In [112]:
groups = {2: g_group,3: pg_group,4: nr_group,5: pg13_group,6: r_group}

In [113]:
result = stats.f_oneway( *groups.values())
result

F_onewayResult(statistic=1.410440902465251, pvalue=0.23734353990419604)

After testing we can see our P value is greater than 0.05 and We reject the alternative hypothesis